# Artificial Neural Networks
#### Geo-Demographic Segmentation

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [ ]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

### Data Loading

In [ ]:
PATH = "../../../../Deep_Learning/ANN/Python/Churn_Modelling.csv"

In [ ]:
dataset = pd.read_csv(PATH)

In [ ]:
dataset.head()

In [ ]:
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

### Encoding the Categorical Variables

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

ct = ColumnTransformer(transformers=[('encoder', 
                                      OneHotEncoder(), [1])], 
                       remainder='passthrough')
X = np.array(ct.fit_transform(X))

### Train Test Split

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.20, 
                                                    random_state=42)

### Feature Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

### K-Fold Cross Validation

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

In [ ]:
def build_classifer():
    tf.random.set_seed(42)
    classifier = Sequential()
    classifier.add(Dense(6, activation='relu', input_shape=(12, )))
    classifier.add(Dense(3, activation='relu'))
    classifier.add(Dense(1, activation='sigmoid'))
    classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return classifier

In [ ]:
epochs = 100
batch_size = 32

classifier = KerasClassifier(build_fn=build_classifer, 
                             batch_size=batch_size, epochs=epochs)
accuracies = cross_val_score(estimator=classifier, X=X_train, 
                             y=y_train, cv=10, n_jobs=-1)

In [ ]:
print("Mean \t\t\t%.2f" % accuracies.mean())
print("Standard Deviation \t%.2f" % accuracies.std())